In [3]:
# various tools

RR = RealField(2048)

from random import shuffle

img = sqrt(-1)

def frange(a,b,s):
    L=[]
    t = a
    while(t<b):
        L.append(t)
        t += s
    return L

def root_Hermite(b):
    b = RR(b)
    return RR(( (b/(2*pi*e))*((pi*b)^(1/b)))^(1/(2*(b-1))))

'''
@return the length of the vectors produced by the short vector sampler
@param d: dimension of the lattice
@param V: the volume of the lattice
@param b0: beta_0
@param b1: beta_1
'''
def short_vector_length(d, V, b0, b1):
    d = RR(d)
    V = RR(V)
    b0 = RR(b0)
    b1 = RR(b1)
    return RR( sqrt(4/3) * (V^(1/d)) * (root_Hermite(b1)^(b1-1)) * (root_Hermite(b0)^(d - b1)) )

def B(alpha, x):
    alpha = RR(alpha)
    x = RR(x)
    if x < 1e-100:
        return RR(1)
    return RR( RR(gamma(RR(alpha+RR(1))) * bessel_J(alpha, x)) / RR((x/RR(2))^alpha) )

In [ ]:
'''
@param T: threshold
@param N: number of short vectors sampled
@param alpha: the parameter of the centered binomial of the LWE oracle
@param q: the size of the field
@param m: the number of rows of the matrix A
@param nlat: the dimension of the lattice part
@param nfft: the length of the lossy source code
@param kfft: the dimension of the lossy source code
@param dlat: the radius of the ball in which the sampled short vectors are drawn uniformly
@param dlsc: the average decoding distance
@param slsc: the standard deviation of the decoding distance
'''

<b>Good Guess</b>

$$
\begin{array}{lcl}
P_\mathsf{good} & := & \mathbb{P}\left( F^{(\mathsf{lsc})}_{\mathbf{s}_{\mathsf{enu}}} \left( \mathbf{G}^{\top} \mathbf{s}_{\mathsf{fft}} \right) \geq T \right) \\
& \approx & 0.5
\end{array}
$$
if
$$
\frac{T}{N} = \frac{\exp\left( \tfrac{-\alpha(\pi \mu_{lsc}/q)^2}{1 + 2 \alpha (\pi \sigma_{lsc}/q)^2}\right)}{\sqrt{1 + 2 \alpha (\pi \sigma_{lsc}/q)^2}} \cdot \int_0^1 \beta_{sieve} \cdot t^{\beta_{sieve} - 1} \cdot e^{-\alpha\left( \frac{\pi d_{lat} t}{q}\right)^2} dt
$$

<b>Wrong Guess</b>

We assume that $(\widetilde{\mathbf{s}_{\mathsf{enu}}}, \widetilde{\mathbf{s}_{\mathsf{fft}}}) \neq (\mathbf{s}_{\mathsf{enu}}, \mathbf{s}_{\mathsf{fft}})$.


$$
\begin{array}{lcl}
P_\mathsf{wrong} & := & \mathbb{P}\left( F^{(\mathsf{lsc})}_{\widetilde{\mathbf{s}_{\mathsf{enu}}}} \left( \mathbf{G}^{\top} \widetilde{\mathbf{s}_{\mathsf{fft}}} \right) \geq T \right) \\
& \approx & \mathbb{P}\left( \mathcal{D} + \mathcal{N}\left(0, \sqrt{N/2}\right) \geq T \right) \\
& \approx & \displaystyle{\int_{\mathbb{R}}} \max_{\substack{i,j \\ N \cdot G(i,j) = T}} \left( \min\left( 1 , \lambda_i \mu_j \right)\right) \cdot \tfrac{e^{-t^2/N}}{\sqrt{\pi N}} dt
\end{array}
$$
where
$$
G(i,j) := B\left( \tfrac{\beta_{\mathsf{sieve}}}{2}, \tfrac{2 \pi}{q} d_{\mathsf{lat}} i\right) \cdot B\left( \tfrac{n_{\mathsf{fft}}}{2} - 1, \tfrac{2 \pi}{q} d_{\mathsf{lsc}} j\right)
$$
and
$$
B(\alpha, x) := \frac{\Gamma(\alpha + 1) J_{\alpha}(x)}{(x/2)^{\alpha}}
$$
and 
$$
d_\mathsf{lsc} \sim \mathcal{N}(\mu_{\mathsf{lsc}}, \sigma_{\mathsf{lsc}})
$$
and
$$
\lambda_i := \frac{2 \cdot \delta(\beta_{\mathsf{bkz}})^{\beta_{\mathsf{sieve}}(m + n_\mathsf{lat} - \beta_{\mathsf{sieve}})} \cdot \pi^{(\beta_{\mathsf{sieve}})/2} \cdot i^{\beta_{\mathsf{sieve}}-1}}{q^{\beta_{\mathsf{sieve}} \cdot \tfrac{m}{m + n_\mathsf{lat}}} \cdot \Gamma\left(\frac{\beta_{\mathsf{sieve}}}{2}\right)} 
$$ 
and
$$
\mu_j := \frac{2 \cdot \pi^{n_{\mathsf{fft}}/2} \cdot j^{n_{\mathsf{fft}}-1}}{q^{k_{\mathsf{fft}}} \cdot \Gamma\left(\frac{n_{\mathsf{fft}}}{2}\right)} 
$$




In [4]:
import pickle

from estimator.estimator.cost import Cost
from estimator.estimator.lwe_parameters import LWEParameters
from estimator.estimator.lwe import Estimate
from estimator.estimator.reduction import delta as deltaf
from estimator.estimator.reduction import RC, ReductionCost
from estimator.estimator.conf import red_cost_model as red_cost_model_default
from estimator.estimator.util import local_minimum, early_abort_range
from estimator.estimator.io import Logging
from estimator.estimator.nd import NoiseDistribution
from estimator.estimator.schemes import (
    Kyber512,
    Kyber768,
    Kyber1024,
)

RR = RealField(2048)

def get_reduction_cost_model(nn):
    matzov_nns={
        "CN": "list_decoding-naive_classical",
        "CC": "list_decoding-classical",
    }
    if nn in matzov_nns:
        return RC.MATZOV.__class__(nn=matzov_nns[nn])
    elif nn == "C0":
        return RC.ADPS16
    else:
        raise Error("unknown cost model '{}'".format(nn))

def cost_sample(m, beta0, beta1, N, red_cost_model):
	rho, T, _, _ = red_cost_model.short_vectors(
		beta = beta0, N=N, d=m, sieve_dim=beta1
	)
	return rho, T
    
C_mul = RR(1024)
C_add = RR(160)

# survival function of D when dlsc is drawn as a normal with mean avg_dlsc and standard deviation sdv_dlsc (approximation)
def sv_D(T, N, q, m, alpha, nenu, nlat, nfft, kfft, beta0, beta1, dlat, avg_dlsc, sdv_dlsc):
    T = RR(T)
    N = RR(N)
    q = RR(q)
    alpha = RR(alpha)
    m = RR(m)
    nlat = RR(nlat)
    nfft = RR(nfft)
    kfft = RR(kfft)
    nenu = RR(nenu)
    n = nlat + nfft + nenu
    beta0=RR(beta0)
    beta1=RR(beta1)
    dlat=RR(dlat)
    avg_dlsc=RR(avg_dlsc)
    sdv_dlsc=RR(sdv_dlsc)
    
    def sv_D_sphere(dlsc):
        dlsc = RR(dlsc)

        # compute the bound of i
        bi_inf = RR(0)
        bi_sup = RR(sqrt(beta1)*q/2)
        bi_cur = RR((bi_inf + bi_sup)/2)
        while bi_sup - bi_inf > 0.000000001:
            if B(RR(beta1/RR(2)),RR(RR(RR(2)*pi*dlat*bi_cur)/RR(q))) >= RR(T/N):
                bi_inf = bi_cur
            else :
                bi_sup = bi_cur
            bi_cur = RR((bi_inf + bi_sup)/RR(2))
        bi = bi_inf
        
        # function to maximize
        def func(ii):
            ii = RR(ii)

            # compute j such that G(i,j) = T/N
            Bi = B(RR(beta1/RR(2)),RR(RR(RR(2)*pi*dlat*ii)/RR(q)))
    
            j_inf = RR(0)
            j_sup = RR(sqrt(nfft)*q/2)
            j_cur = RR((j_inf + j_sup)/2)
            while j_sup - j_inf > 0.000000001:
                if Bi * B(RR(nfft/RR(2))-RR(1),RR(RR(RR(2)*pi*dlsc*j_cur)/RR(q))) >= RR(T/N):
                    j_inf = j_cur
                else :
                    j_sup = j_cur
                j_cur = RR((j_inf + j_sup)/RR(2))
            jj = j_inf
            
            # compute P(N_(i,j) > 0)
            VolSphere_i = RR(1) + RR(beta1/2) * RR(log(pi, 2)) + RR(beta1-1)*RR(log(RR(ii), 2)) - RR(log(RR(gamma(RR(beta1/2))), 2))
            VolSphere_j = RR(1) + RR(nfft/2) * RR(log(pi, 2)) + RR(nfft-1)*RR(log(RR(jj), 2)) - RR(log(RR(gamma(RR(nfft/2))), 2))
            VolLat_lat = RR(beta1*nlat/(m+nlat)) * RR(log(RR(q),2)) + RR(beta1*(m+nlat-beta1)) * RR(log(RR(root_Hermite(beta0)), 2))
            VolLat_lsc = RR(nfft - kfft)*RR(log(RR(q), 2))
            li = RR(VolSphere_i + VolLat_lat - beta1*RR(log(RR(q), 2)))
            mj = RR(VolSphere_j + VolLat_lsc - nfft*RR(log(RR(q),2)))
            #mj = RR(log(RR(2) * RR(RR(pi)^RR(nfft/2)) * RR(RR(jj)^RR(nfft - 1)) / RR((RR(q)^RR(kfft)) * RR(gamma(RR(nfft/2)))), 2))
            #li = RR(log(RR(2) * RR(RR(root_Hermite(beta0))^RR(beta1*(m+nlat-beta1))) * RR(RR(pi)^RR(beta1/2)) * RR(RR(ii)^RR(beta1 - 1)) / RR(RR(q)^RR(beta1*m/(m+nlat)) * RR(gamma(RR(beta1/2)))) , 2))
            return float(min(0, mj + li))

        return find_local_maximum(func, 0, bi)[0]

    #return RR(log(numerical_integral(lambda x: RR(exp(RR(-0.5 * ((x-avg_dlsc)/sdv_dlsc)^2)) / RR(sdv_dlsc*sqrt(2*pi))) * RR(2^RR(sv_D_sphere(x))), avg_dlsc - 3*sdv_dlsc, avg_dlsc + 3*sdv_dlsc)[0], 2))
    res = 0
    norm = 0
    step = RR((3.0*sdv_dlsc)/12.0)
    dlsc = RR(avg_dlsc-3.0*sdv_dlsc)
    while dlsc <= avg_dlsc+3.0*sdv_dlsc+0.0000001:
        prob = RR(exp(RR(-0.5 * ((dlsc-avg_dlsc)/sdv_dlsc)^RR(2))) / RR(sdv_dlsc*sqrt(2*pi)))
        norm += RR(prob * step)
        res += RR(step * prob * RR(2)^RR(sv_D_sphere(dlsc)))
        dlsc += RR(step)
    return float(log(res/norm, 2))    

In [67]:

file_name = "optimized_withExperimentalPolar.pkl"

with open(file_name, 'rb') as handle:
		results = pickle.load(handle)

nn = "CN"
sch = Kyber1024
parameters = results[sch][nn] # Parameters for the scheme "scheme" with cost model "nn"

#print(parameters)

print("Complexity = ", float(log(parameters['complexity'], 2)))

q = RR(3329)
alpha = (RR(3) if sch == Kyber512 else RR(2))
m = RR(parameters['m'])
beta0 = RR(parameters['beta0'])
beta1 = RR(parameters['beta1'])
nlat = RR(parameters['nlat'])
nfft = RR(parameters['nfft'])
kfft = RR(parameters['kfft'])
nenu = RR(parameters['nenu'])
n = RR(nlat + nfft + nenu)

N = RR(parameters['N'])
Threshold = RR(parameters['treshold'])

dlat = RR(parameters['dlat'])
avg_dlsc = RR(parameters['avg_dlsc'])
sdv_dlsc = RR(parameters['sdv_dlsc'])


print("q =", int(q))
print("alpha =", int(alpha))
print("n =", int(n))
print("m =", int(m))
print("beta0 =", int(beta0))
print("beta1 =", int(beta1))
print("nenu =", int(nenu))
print("nfft =", int(nfft))
print("kfft =", int(kfft))
print("nlat =", int(nlat))
print("dlat =", dlat.n())
print("avg_dlsc =", avg_dlsc.n())
print("sdv_dlsc =", sdv_dlsc.n())
print("N = ", float(log(N, 2)))
print("T =", float(log(Threshold, 2)))

print("sqrt(4/3)^beta1 = ", RR(log(sqrt(4/3)^beta1, 2)).n())
R = RR(parameters['R'])
print("R = ", float(log(R, 2)))
print("T_sample = ", float(log(RR(parameters['T_sample']), 2)))
print("N * T_decode = ", float(log(RR(parameters['NT_decode']), 2)))
print("T_fft = ", float(log(RR(parameters['T_FFT']), 2)))

Complexity =  251.19354313893066
q = 3329
alpha = 2
n = 1024
m = 789
beta0 = 807
beta1 = 788
nenu = 29
nfft = 103
kfft = 16
nlat = 892
dlat = 4967.88797645771
avg_dlsc = 2411.83511579016
sdv_dlsc = 42.0153691099208
N =  163.5053760362224
T = 85.39767499992772
sqrt(4/3)^beta1 =  163.524774715864
R =  42.03608747908647
T_sample =  250.88402017094222
N * T_decode =  201.7205629948851
T_fft =  206.74103402280937


In [69]:
sv_D_ = sv_D(Threshold, N, q, m, alpha, nenu, nlat, nfft, kfft, beta0, beta1, dlat, avg_dlsc, sdv_dlsc)
print("sv_D(T) = ", sv_D_)
sv_N_ = log(RR(RR(RR(1) - RR(erf(RR(RR(Threshold)/RR(sqrt(N))))))/RR(2)), 2)
print("sv_N(T) = ", sv_N_.n())
print("Pwrong = ", max(sv_N_, sv_D_).n())
nb_fp = RR((2^max(sv_N_, sv_D_))*R*(q^kfft))
print("R q^kfft Pwrong = ", nb_fp.n())

sv_D(T) =  -285.13910831449857
sv_N(T) =  -231.250057961331
Pwrong =  -231.250057961331
R q^kfft Pwrong =  0.250000000000000


Estimation of the smallest vector in the global lattice seen as a random lattice:
$$
\lambda_1\left( q \Lambda\left( \mathbf{B}_{\mathsf{global}}\right)^\vee + \mathbf{r}_{\mathsf{proj}} \right) \approx \frac{q}{ V_{\mathsf{global}}^{\frac{1}{\beta_{\mathsf{sieve}} + n_{\mathsf{fft}}}}}\cdot \sqrt{\tfrac{\beta_{\mathsf{sieve}} + n_{\mathsf{fft}}}{2 \pi e}}
$$
The shortest vector accross $R \cdot q^{k_{\mathsf{fft}}}$ cosets is:
$$
\approx \frac{\lambda_1\left( q \Lambda\left( \mathbf{B}_{\mathsf{global}}\right)^\vee + \mathbf{r}_{\mathsf{proj}} \right)}{\left( R \cdot q^{k_{\mathsf{fft}}} \right)^{\frac{1}{\beta_{\mathsf{sieve}} + n_{\mathsf{fft}}}}}
$$

Length of the target vector:
$$
\approx \sqrt{\tfrac{\alpha (\beta_{\mathsf{sieve}} + n_{\mathsf{fft}})}{2}} 
$$

In [68]:
VolLat_lat = RR(beta1*nlat/(m+nlat)) * RR(log(RR(q),2)) + RR(beta1*(m+nlat-beta1)) * RR(log(RR(root_Hermite(beta0)), 2))
VolLat_lsc = RR(nfft - kfft)*RR(log(RR(q), 2))
V = RR(2^(VolLat_lat+VolLat_lsc))
lambda1 = q * (V^(-1/(beta1+nfft))) * sqrt((beta1 + nfft)/(2*pi*e))
target = sqrt(alpha*(beta1+nfft)/2)
print("target = ", target.n())
print("lambda1 = ", lambda1.n())
print("lambda1/target = ", RR(lambda1/target).n())
r = RR(R*(q^kfft))^RR(1/(beta1+nfft))
# r < lambda1/target ?
print("r = (R q^kfft)^(1/(beta1+nfft)) = ", r.n())
print("lambda1/r = ", RR(lambda1/r).n())

target =  29.8496231131986
lambda1 =  36.4481881852338
lambda1/target =  1.22106024746147
r = (R q^kfft)^(1/(beta1+nfft)) =  1.19523577776885
lambda1/r =  30.4945592017601


<b> What follows is a draft.</b>

We plug the parameters of Matzov. We replace the modulo switching by our code method with
$$
q^{k_{\mathsf{fft}}} = p^{n_{\mathsf{fft}}}
$$
and replace the Prange bet by the enumeration method of Matzov:
$$
R = 2^{n_{\mathsf{enu}} H(\mathcal{B}_{\alpha})}
$$

In [70]:
#Matzov_parameters = [kyber, nn, q, n, p, m, nenu, nfft, beta0, beta1]

# the red cost metrics are wrong but it does not matter
# Matzov Table 3
Matzov_parameters = [Kyber512, "CC", 3329, 512, 5, 474, 19, 34, 377, 380]
#Matzov_parameters = [Kyber768, "CC", 3329, 768, 4, 659, 30, 59, 576, 570]
#Matzov_parameters = [Kyber1024, "CC", 3329, 1024, 4, 836, 40, 82, 809, 790]
# Matzov Table 4
#Matzov_parameters = [Kyber512, "CC", 3329, 512, 6, 474, 19, 30, 380, 378]
#Matzov_parameters = [Kyber768, "CC", 3329, 768, 4, 655, 31, 58, 580, 566]
#Matzov_parameters = [Kyber1024, "CC", 3329, 1024, 4, 839, 39, 82, 816, 786]
# Matzov Table 5
#Matzov_parameters = [Kyber512, "CC", 3329, 512, 5, 485, 17, 33, 379, 383]
#Matzov_parameters = [Kyber768, "CC", 3329, 768, 5, 652, 27, 51, 580, 574]
#Matzov_parameters = [Kyber1024, "CC", 3329, 1024, 4, 834, 36, 82, 813, 794]


sch = Matzov_parameters[0]
nn = Matzov_parameters[1]
alpha = (RR(3) if sch == Kyber512 else RR(2))
q = RR(Matzov_parameters[2])
n = RR(Matzov_parameters[3])
p = RR(Matzov_parameters[4])
m = RR(Matzov_parameters[5])
nenu = RR(Matzov_parameters[6])
nfft = RR(Matzov_parameters[7])
beta0 = RR(Matzov_parameters[8])
beta1 = RR(Matzov_parameters[9])
nlat = RR(n - nfft - nenu)

kfft = nfft * RR(log(RR(p),q))

def entropy_enu(nenu, alpha):
	# Centered Binomial distribution
	assert(alpha == 2 or alpha==3)
	prob_B2=[6/16, 4/16, 1/16]
	prob_B3=[20/64, 15/64, 6/64, 1/64]
	if alpha == 2:
		return RR(nenu * (-prob_B2[0]*log(prob_B2[0],2)-2*prob_B2[1]*log(prob_B2[1],2)-2*prob_B2[2]*log(prob_B2[2],2)))
	else:
		return RR(nenu * (-prob_B3[0]*log(prob_B3[0],2)-2*prob_B3[1]*log(prob_B3[1],2)-2*prob_B3[2]*log(prob_B3[2],2)-2*prob_B3[3]*log(prob_B3[3],2)))

def entropy_enu(nenu, alpha):
    stddev = RR(sqrt(RR(alpha/2)))
    return nenu * RR((1 / 2 + log(sqrt(2 * pi) * stddev)) / log(2.0))

R = RR(2^entropy_enu(nenu, alpha))
mu = RR(0.5)

def Nf(q, p, m, nenu, nfft, nlat, beta0, beta1):
    stddev = RR(sqrt(alpha/2))
    lsigma_s = RR(
        RR(stddev ^ RR(m / (m + nlat)))
        * RR((stddev * q) ^ RR(nlat / (m + nlat)))
        * RR(sqrt(RR(4 / 3.0)))
        * RR(sqrt(RR(beta1 / 2 / pi / e)))
        * RR(root_Hermite(beta0) ^ RR(m + nlat - beta1))
    )
    N = RR(
        RR(exp(RR(4 * (lsigma_s * pi / q) ^ 2)))
        * RR(exp(RR(nfft / 3.0 * (stddev * pi / p) ^ 2)))
        * RR(entropy_enu(nenu, alpha) + nfft * log(p) + log(1 / mu))
    )

    return N
    
N = Nf(q, p, m, nenu, nfft, nlat, beta0, beta1)
Threshold = sqrt(2)*RR(erfinv(RR(RR(1.0) - RR(mu/RR(RR(2 ^ entropy_enu(nenu, alpha)) * RR(p ^ nfft)))))) * RR(sqrt(RR(0.5 * N)))


avg_dlat = RR(short_vector_length(RR(m+nlat), RR(q)^RR(nlat), RR(beta0), RR(beta1)))
dlat = RR(avg_dlat * (beta1+RR(1)) / beta1)
avg_dlsc = RR(q^(RR(1 - kfft/nfft))) * RR(sqrt(RR(nfft/2/pi/e)))
sdv_dlsc = RR(sqrt(avg_dlsc/2))


print("q =", int(q))
print("p =", int(p))
print("alpha =", int(alpha))
print("n =", int(n))
print("m =", int(m))
print("beta0 =", int(beta0))
print("beta1 =", int(beta1))
print("nenu =", int(nenu))
print("nfft =", int(nfft))
print("kfft =", int(kfft))
print("nlat =", int(nlat))
print("avg_dlat =", avg_dlat.n())
print("dlat =", dlat.n())
print("avg_dlsc =", avg_dlsc.n())
print("sdv_dlsc =", sdv_dlsc.n())
print("N = ", float(log(N, 2)))
print("T =", float(log(Threshold, 2)))
print("sqrt(4/3)^beta1 = ", RR(log(sqrt(4/3)^beta1, 2)).n())
print("R = ", float(log(R, 2)))

def cost_FFT(q,kfft):
	C_mul = RR(1024)
	C_add = RR(160)
	return RR(C_mul*RR(115928) + C_add*RR(240500))*RR(kfft)*RR(q)**(RR(kfft - RR(1)))


def cost_decode(q,nfft,L_size=1):
	C_mul = RR(1024)
	C_add = RR(160)
	L_size = RR(L_size)
	nfft_ = RR(2)**(floor(log(RR(nfft), RR(2))) + RR(1))
	return RR(3) * L_size * (C_mul * RR(115928) + C_add * RR(240500)) * RR(nfft_) * log(RR(nfft_), RR(2))

T_FFT = cost_FFT(q,kfft)
rho, T_sample = cost_sample(m + nlat, beta0, beta1, N, get_reduction_cost_model(nn))
avg_dlat = RR(rho * RR(q**RR(nlat/(m+nlat))) * RR(root_Hermite(beta0)**RR(m+nlat-RR(1))))
print("avg_dlat (Martin) = ", avg_dlat.n())
print("dlat (Martin) = ", float(avg_dlat * (beta1+RR(1)) / beta1))
T_decode = N*cost_decode(q,nfft)
T_total = T_sample + R*(T_decode + T_FFT)
print("T_sample = ", float(log(T_sample, 2)))
print("N * T_decode = ", float(log(T_decode, 2)))
print("T_fft = ", float(log(T_FFT, 2)))
print("T_total = ", float(log(T_total, 2)))
print("Nb target = ", RR(log(R*(q^kfft), 2)).n())
VolLat_lat = RR(beta1*nlat/(m+nlat)) * RR(log(RR(q),2)) + RR(beta1*(m+nlat-beta1)) * RR(log(RR(root_Hermite(beta0)), 2))
VolLat_lsc = RR(nfft - kfft)*RR(log(RR(q), 2))
V = RR(2^(VolLat_lat+VolLat_lsc))
lambda1 = q * (V^(-1/(beta1+nfft))) * sqrt((beta1 + nfft)/(2*pi*e))
print("lambda1 = ", lambda1.n())
r = RR(R*(q^kfft))^RR(-1/(beta1+nfft))
print("r = ", r.n())
print("r*lambda1 = ", float(r*lambda1))
print("target = ", sqrt(alpha*(beta1+nfft)/2).n())


print("\nQue sur lat")
V = 2^VolLat_lat
lambda1 = q * (V^(-1/(beta1))) * sqrt((beta1)/(2*pi*e))
print("lambda1 = ", lambda1.n())
r = RR(R*(q^kfft))^RR(-1/beta1)
print("r = ", r.n())
print("r*lambda1 = ", float(r*lambda1))
print("target = ", sqrt(alpha*(beta1)/2).n())

q = 3329
p = 5
alpha = 3
n = 512
m = 474
beta0 = 377
beta1 = 380
nenu = 19
nfft = 34
kfft = 6
nlat = 459
avg_dlat = 2932.59984741281
dlat = 2940.31721543231
avg_dlsc = 939.390306053978
sdv_dlsc = 21.6724514770939
N =  81.39815525640167
T = 43.89037217487528
sqrt(4/3)^beta1 =  78.8571248629803
R =  44.451959875283166
avg_dlat (Martin) =  2932.59984741293
dlat (Martin) =  2940.31721543244
T_sample =  139.11085934051616
N * T_decode =  118.79601695354943
T_fft =  97.2268606166686
T_total =  163.2479773707
Nb target =  123.397515101453
lambda1 =  30.1710807483576
r =  0.813344452020989
r*lambda1 =  24.5394811381539
target =  24.9198715887542

Que sur lat
lambda1 =  29.4366374398395
r =  0.798447634035464
r*lambda1 =  23.503613517799618
target =  23.8746727726266


In [108]:
sv_D_ = sv_D(Threshold, N, q, m, alpha, nenu, nlat, nfft, kfft, beta0, beta1, dlat, avg_dlsc, sdv_dlsc)
print("sv_D(T) = ", sv_D_.n())
sv_N_ = log(RR(RR(RR(1) - RR(erf(RR(RR(Threshold)/RR(sqrt(N))))))/RR(2)), 2)
print("sv_N(T) = ", sv_N_.n())
print("Pwrong = ", max(sv_N_, sv_D_).n())
nb_fp = RR((2^max(sv_N_, sv_D_))*R*(q^kfft))
print("R q^kfft Pwrong = ", nb_fp.n())

sv_D(T) =  -127.964495407214
sv_N(T) =  -125.397515101453
Pwrong =  -125.397515101453
R q^kfft Pwrong =  0.250000000000001


In [109]:
def sigma(n, logT):
    # Number of dual vectors from a full sieve equals `(4/3)^{n/2}`.
    N = (4./3)**(n/2.)
    # The dual vectors have a length concentrated around `sqrt(4/3)*GH(n)`.
    ell = sqrt((n)/(2*pi*e)) * sqrt(4./3) #* q * (V^(-1/(n)))
    # The minimal advantage that we must have to have the attack work, must satisfy:
    # `log(T) <= N epsilon^2.
    epsilon = sqrt(logT / N)
    # We have `epsilon = exp(-2pi^2 sigma^2 ell^2)`, giving an upper bound on sigma.
    sigma = sqrt(.5 * log(1.0/epsilon)) / (pi * ell)
    return RR(sigma)

V = 2^VolLat_lat
lambda1 = sqrt((beta1)/(2*pi*e))
print("lambda1 = ", lambda1.n())
sigma_ = sigma(beta1, log(RR(R*(q^kfft)), 2))
print("sigma = ", sigma_.n())
print("target = ", sigma_*sqrt(beta1).n())
r = RR(R*(q^kfft))^RR(-1/beta1)
print("r = ", r.n())
print("r*lambda1 = ", float(r*lambda1))

lambda1 =  4.71687777870503
sigma =  0.206301972399819
target =  4.02156579581898
r =  0.798447634035464
r*lambda1 =  3.766179902441485


In [31]:
# Recompute Complexity

def compute_p0(alpha):
	# Centered Binomial distribution
	assert(alpha == 2 or alpha==3)
	prob_B2=[6/16, 4/16, 1/16]
	prob_B3=[20/64, 15/64, 6/64, 1/64]
	if alpha == 2:
		p0 = prob_B2[0]
	else:
		p0 = prob_B3[0]
	return RR(p0)

def cost_FFT(q,kfft):
	C_mul = RR(1024)
	C_add = RR(160)
	return RR(C_mul*RR(115928) + C_add*RR(240500))*RR(kfft)*RR(q)**(RR(kfft - RR(1)))


def cost_decode(q,nfft,L_size=1):
	C_mul = RR(1024)
	C_add = RR(160)
	L_size = RR(L_size)
	nfft_ = RR(2)**(floor(log(RR(nfft), RR(2))) + RR(1))
	return RR(3) * L_size * RR(C_mul*RR(115928) + C_add*RR(240500)) * RR(nfft_) * log(RR(nfft_), RR(2))


p0 = RR(compute_p0(alpha))
R = max(RR(1), RR(2 * (p0**(-nenu))))

Threshold = RR(N)
Threshold *= RR(beta1 * numerical_integral(lambda t: RR(RR(t**RR(beta1 - 1)) * RR(exp(RR(-alpha*(pi*t*dlat/q)**2)))), 0, 1)[0])
Threshold *= RR(exp(RR(-alpha * (pi*avg_dlsc/q)**2 / (1 + 2*alpha*(pi*sdv_dlsc/q)**2))))
Threshold /= RR(sqrt(1 + 2*alpha*(pi*sdv_dlsc/q)**2))

survival_normal = RR(RR(RR(1) - RR(erf(RR(RR(Threshold)/RR(sqrt(N))))))/RR(2))
print("survival normal = ", log(survival_normal, 2).n())
target_fp = RR(survival_normal*R*(q^kfft))
print("R q^kfft Pwrong = ", target_fp.n())


T_FFT = cost_FFT(q,kfft)
rho, T_sample = cost_sample(m + nlat, beta0, beta1, N, get_reduction_cost_model(nn))
avg_dlat = RR(rho * RR(q**RR(nlat/(m+nlat))) * RR(root_Hermite(beta0)**RR(m+nlat-RR(1))))
avg_dlat_ = RR(short_vector_length(RR(m+nlat), RR(q)**RR(nlat), RR(beta0), RR(beta1)))
print("avg_dlat (Martin) = ", avg_dlat.n())
print("avg_dlat (Matzov) = ", avg_dlat_.n())
print("dlat = ", float(avg_dlat * (beta1+RR(1)) / beta1))
T_decode = N*cost_decode(q,nfft)
T_total = T_sample + R*(T_decode + T_FFT)

print("N = ", float(log(N, 2)))
print("sqrt(4/3)^beta1 = ", RR(log(sqrt(4/3)^beta1, 2)).n())
print("T =", float(log(Threshold, 2)))
print("R = ", float(log(R, 2)))
print("T_sample = ", float(log(T_sample, 2)))
print("N * T_decode = ", float(log(T_decode, 2)))
print("T_fft = ", float(log(T_FFT, 2)))
print("T_total = ", float(log(T_total, 2)))

survival normal =  -96.6069852411221
R q^kfft Pwrong =  0.250000000000001
avg_dlat (Martin) =  3005.10909812907
avg_dlat (Matzov) =  3005.10909812907
dlat =  3012.60313578026
N =  78.51934117788555
sqrt(4/3)^beta1 =  83.2150186054082
T = 42.25529800521065
R =  1.0
T_sample =  117.092
N * T_decode =  114.24910130937802
T_fft =  112.13404878168728
T_total =  117.51757699318861


In [ ]:
# Volume of the sublattice after BKZ
from sage.modules.free_module_integer import IntegerLattice

m = 30
nlat = 30
Bbkz = 30
Bsieve = 30

volume_sublattice_total = 0
nb_tests = 1
for i in range(nb_tests):
    Alat_tab = [[randrange(q) for c in range(nlat)] for r in range(m)]
    Alat = matrix(ZZ, Alat_tab)
    Blat = (identity_matrix(m).stack(Alat.transpose())).augment( matrix(ZZ, m, nlat, 0).stack(q*identity_matrix(nlat)) )
    L = IntegerLattice(Blat.transpose(), lll_reduce=False)
    #print(L.volume())
    Lbkz = L.BKZ(block_size=Bbkz)
    sub_Lbkz = IntegerLattice(Lbkz.C.matrix_from_rows(range(Bsieve)), lll_reduce=False)
    
    volume_sublattice_total += float(sub_Lbkz.volume())

print(float(L.volume()^RR(Bsieve/(m+nlat)) * root_Hermite(Bbkz)^(Bsieve*(m+nlat-Bsieve))))
print(float(volume_sublattice_total/nb_tests))